In [28]:
import pandas as pd 
from backtest.backtest_check import backtester
import os 

list_of_stats = []
df = pd.read_pickle(r"C:\Users\sachi\OneDrive\Documents\BOTS\nnfx_bot\nepse\nepse_pickles\ACLBSL_2024-08-01.pkl")
df.rename(columns={"Date": "Time", "Ltp": "Close"}, inplace=True)
df['Open'] = df['Open'].str.replace(',', '').astype(float)
df['High'] = df['High'].str.replace(',', '').astype(float)
df['Low'] = df['Low'].str.replace(',', '').astype(float)
df['Close'] = df['Close'].str.replace(',', '').astype(float)
stocks_df = backtester().strategy(df)
stats = backtester().stock_backtesting(stocks_df,name='ACLBSL')
list_of_stats.append(stats)
pd.DataFrame(list_of_stats)

BUY signal: 10 units at 882.1 each, invested size: 8821.0
SELL signal: Sold 10 units at 813.5 each, total selling price: 8135.0, loss: -686.0
BUY signal: 10 units at 745.0 each, invested size: 7450.0
BUY signal: 10 units at 724.0 each, invested size: 7240.0
BUY signal: 10 units at 625.5 each, invested size: 6255.0
SELL signal: Sold 30 units at 559.0 each, total selling price: 16770.0, loss: -4175.0
SELL signal received, but no positions to sell.
BUY signal: 10 units at 535.0 each, invested size: 5350.0
SELL signal: Sold 10 units at 540.0 each, total selling price: 5400.0, profit: 50.0
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, bu

,Name,Total Gain/Loss,Number of Positive Trades,Number of Negative Trades,Win Percent
0,ACLBSL,-5861.0,1,3,25.0


In [29]:

folder_path = r"C:\Users\sachi\OneDrive\Documents\BOTS\nnfx_bot\nepse\nepse_pickles"

list_of_stats = []

for file in os.listdir(folder_path):
    if file.endswith(".pkl"):
        path = os.path.join(folder_path, file)
        try:
            df = pd.read_pickle(path)  # Use the variable path without raw string
            df.rename(columns={"Date": "Time", "Ltp": "Close"}, inplace=True)
            df['Open'] = df['Open'].str.replace(',', '').astype(float)
            df['High'] = df['High'].str.replace(',', '').astype(float)
            df['Low'] = df['Low'].str.replace(',', '').astype(float)
            df['Close'] = df['Close'].str.replace(',', '').astype(float)
            stocks_df = backtester().strategy(df)
            name = file.split('_')[0]
            stats = backtester().stock_backtesting(df=df,name=name)
            list_of_stats.append(stats)
        except Exception as e:
            print(f"Error reading {file}: {e}")

stats_df = pd.DataFrame(list_of_stats)

stats_df.to_csv('stats.csv')

BUY signal: 10 units at 882.1 each, invested size: 8821.0
SELL signal: Sold 10 units at 813.5 each, total selling price: 8135.0, loss: -686.0
BUY signal: 10 units at 745.0 each, invested size: 7450.0
BUY signal: 10 units at 724.0 each, invested size: 7240.0
BUY signal: 10 units at 625.5 each, invested size: 6255.0
SELL signal: Sold 30 units at 559.0 each, total selling price: 16770.0, loss: -4175.0
SELL signal received, but no positions to sell.
BUY signal: 10 units at 535.0 each, invested size: 5350.0
SELL signal: Sold 10 units at 540.0 each, total selling price: 5400.0, profit: 50.0
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, bu

In [31]:
stats_df['Total Gain/Loss'].sum()

-140827.5

In [2]:
stocks_df = backtester().strategy(df)


In [3]:
from src.chart import create_figure_sig

In [4]:
create_figure_sig(stocks_df)

In [5]:
import pandas as pd

def stock_backtesting(df, unit=10, name=None):
    position_size = unit
    buy_list = []
    all_position = []
    gain_or_loss_list = []
    positive_count = 0
    negative_count = 0
    stats_dict = {}
    name = name

    for index, row in df.iterrows():
        if row['FINAL_SIGNAL'] == "BUY":
            invested_size = position_size * row['Prev_open']
            buy_list.append(invested_size)
            all_position.append(position_size)
            print(f"BUY signal: {position_size} units at {row['Prev_open']} each, invested size: {invested_size}")
        elif row['FINAL_SIGNAL'] == "SELL":
            if all_position:
                total_invested = sum(buy_list)
                total_position = sum(all_position)
                selling_price = row['Close'] * total_position
                profit_or_loss = selling_price - total_invested
                gain_or_loss_list.append(profit_or_loss)

                buy_list = []
                all_position = []
                if profit_or_loss > 0:
                    positive_count += 1
                    print(f"SELL signal: Sold {total_position} units at {row['Close']} each, total selling price: {selling_price}, profit: {profit_or_loss}")
                elif profit_or_loss < 0:
                    negative_count += 1
                    print(f"SELL signal: Sold {total_position} units at {row['Close']} each, total selling price: {selling_price}, loss: {profit_or_loss}")
            else:
                print("SELL signal received, but no positions to sell.")

    # Calculate overall statistics
    total_gain_or_loss = sum(gain_or_loss_list)
    total_trades = positive_count + negative_count
    win_percent = (positive_count / total_trades) * 100 if total_trades > 0 else 0

    # Store the statistics in stats_dict
    stats_dict['Name'] = name
    stats_dict['Total Gain/Loss'] = total_gain_or_loss
    stats_dict['Number of Positive Trades'] = positive_count
    stats_dict['Number of Negative Trades'] = negative_count
    stats_dict['Win Percent'] = win_percent


    # Print the overall statistics
    print(f"Overall gain/loss: {total_gain_or_loss}")
    print(f"Number of positive trades: {positive_count}")
    print(f"Number of negative trades: {negative_count}")
    print(f"Win Percent: {win_percent}")

    return stats_dict



In [6]:
stats = stock_backtesting(stocks_df, name="Check")

BUY signal: 10 units at 882.1 each, invested size: 8821.0
SELL signal: Sold 10 units at 813.5 each, total selling price: 8135.0, loss: -686.0
BUY signal: 10 units at 745.0 each, invested size: 7450.0
BUY signal: 10 units at 724.0 each, invested size: 7240.0
BUY signal: 10 units at 625.5 each, invested size: 6255.0
SELL signal: Sold 30 units at 559.0 each, total selling price: 16770.0, loss: -4175.0
SELL signal received, but no positions to sell.
BUY signal: 10 units at 535.0 each, invested size: 5350.0
SELL signal: Sold 10 units at 540.0 each, total selling price: 5400.0, profit: 50.0
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
SELL signal received, bu

In [58]:
df = pd.read_pickle(r"C:\Users\sachi\OneDrive\Documents\BOTS\nnfx_bot\yfinance_check\stocks_datas\BAC_data.pkl")
df.reset_index(inplace=True)
btd = backtester().strategy(df)
btd['Prev_open'] = btd['Open'].shift(-1)
btd

,Date,Open,High,Low,Close,Adj Close,Volume,baseline,smaHigh,smaLow,...,macd,std,bb_upper,bb_lower,t1,e1,trendUp,trendDown,FINAL_SIGNAL,Prev_open
0,1973-02-21,4.625000,4.625000,4.625000,4.625000,1.561252,99200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None,4.640625
1,1973-02-22,4.640625,4.640625,4.640625,4.640625,1.566527,47200,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None,4.625000
2,1973-02-23,4.625000,4.625000,4.625000,4.625000,1.561252,133600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None,4.625000
3,1973-02-26,4.625000,4.625000,4.625000,4.625000,1.561252,24000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None,4.625000
4,1973-02-27,4.625000,4.625000,4.625000,4.625000,1.561252,41600,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,None,4.625000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12963,2024-07-18,43.619999,44.240002,42.930000,43.009998,43.009998,48549200,42.44625,42.472,41.537,...,1.013896,1.489680,45.425611,39.466889,5.272321,5.958722,5.272321,0.000000,None,42.860001
12964,2024-07-19,42.860001,43.560001,42.689999,42.900002,42.900002,43903000,42.63125,42.732,41.777,...,1.034325,1.530778,45.692807,39.569693,3.064427,6.123114,3.064427,0.000000,None,42.639999
12965,2024-07-22,42.639999,42.869999,41.959999,42.299999,42.299999,42838300,42.70125,42.915,41.930,...,1.019228,1.502721,45.706691,39.695809,-2.264656,6.010883,0.000000,2.264656,None,42.220001
12966,2024-07-23,42.220001,42.779999,42.150002,42.410000,42.410000,36705100,42.77625,43.010,42.085,...,1.006609,1.494516,45.765282,39.787218,-1.892733,5.978064,0.000000,1.892733,None,42.279999


In [7]:
from nepse.nepsesoup import nepse_ticker_fetcher

In [8]:
nepse_ticker_fetcher()

[]

In [36]:
stats = stock_backtesting(btd)
print("Stats:", stats)


SELL signal received, but no positions to sell.
BUY signal: 10 units at 0.0 each, invested size: 0.0
SELL signal: Sold 10 units at 4.578125 each, total selling price: 45.78125, profit: 45.78125
SELL signal received, but no positions to sell.
SELL signal received, but no positions to sell.
BUY signal: 10 units at 0.0 each, invested size: 0.0
SELL signal: Sold 10 units at 4.9375 each, total selling price: 49.375, profit: 49.375
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
SELL signal: Sold 20 units at 4.015625 each, total selling price: 80.3125, profit: 80.3125
SELL signal received, but no positions to sell.
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY signal: 10 units at 0.0 each, invested size: 0.0
BUY sig

In [67]:
pd.DataFrame([stats])

,Name,Total Gain/Loss,Number of Positive Trades,Number of Negative Trades,Win Percent
0,BBC,-13077.0,2,12,14.285714
